# DegreeDiscountIC (a.k.a the ORACLE)

In [104]:
import networkx as nx
import numpy as np
from numpy.random import default_rng
rng = default_rng(12345)
import matplotlib.pyplot as plt
import sys; sys.path.append("model-independent-cascade\src")
from influence import influence_count

Generating a random graph to test-run the model

In [105]:
G = nx.fast_gnp_random_graph(600, 0.1, directed=True)
edges = list(G.edges())

In [106]:
def sample_edges(edges, C, P_hat, O):
    epo = [(edges[i], P_hat[i], O[i]) for i in range(len(edges))]
    E = [[] for i in range(10)]
    for i in range(len(edges)):
        index = int(epo[i][1]/0.1)
        E[index].append(epo[i])
    E_hat = []
    for i in range(10):
        E[i].sort(key=lambda tuple: tuple[2], reverse=True)
        c_i = int(len(E[i])/(len(edges))*C)+1
        E_hat = E_hat + E[i][:c_i]

    return E_hat

In [107]:
def degdisIC(G, k, P_hat):
    S = set({})
    temp = list(nx.degree(G))
    dd = d = [d for (_,d) in temp]
    t = [0 for i in range(len(d))]
    p = 0.01

    for _ in range(k):
        u = np.argmax(dd)
        dd[u] = -1
        S.add(u)
        neighbors = set(G.neighbors(u))
        for v in (neighbors-S):
            t[v] += 1
            dd[v] = d[v] - 2*t[v] - (d[v] - t[v])*t[v]*p

    return list(S)

## The Algorithm

Coding it step-by-step:

Assigning the _ground-truth_ edge features and initializing the embeddings

In [108]:
for e in edges:
    n1, n2 = e
    alpha_t = rng.uniform(size=4)
    alpha_t /= np.linalg.norm(alpha_t)
    beta_t = rng.uniform(size=4)
    beta_t /= np.linalg.norm(beta_t)
    
    G.edges[n1,n2]['xt'] = np.concatenate((alpha_t, beta_t))

    alpha = rng.uniform(size=4)
    alpha /= np.linalg.norm(alpha)
    beta = rng.uniform(size=4)
    beta /= np.linalg.norm(beta)
    
    G.edges[n1,n2]['x'] = np.concatenate((alpha, beta))


In [110]:
sigma = 1
C = 80
z = 0.95
period = 20 
P_hat = [0 for _ in range(len(edges))]  
O = [0 for _ in range(len(edges))]

In [114]:
Cedges = [a for (a,b,c) in sample_edges(edges, C, P_hat, O)]

In [112]:
f = lambda x1,x2: np.exp(-1)*np.linalg.norm(x1-x2)

In [113]:
Cedges[0]

((0, 15), 0, 0)